In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation as cv
from xgboost.sklearn import XGBClassifier

/Users/nodakanma/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#不要なPassengerId,Cabinを削除。カテゴリー変数を数値に。Embarkedの穴を埋める。目的変数を取り出す。
df = pd.read_csv("train.csv")
y = df["Survived"].as_matrix()
df = df.drop(["PassengerId","Cabin"], axis = 1)
df = df.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
df = df.fillna(value = {'Embarked': 1})

#敬称ごとに変数を追加
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append("4")
df['honor'] = np.array(honor)

#家族の人数を追加
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

#年齢の欠損を敬称別の中央値で埋める。
df = df.fillna(value = {"Age":-1})
a = df.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df[df["honor"]==i[1]]["Age"].median()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df = df.drop(["Age","honor"], axis = 1)
df = pd.concat([df, frame], axis=1)

In [4]:
#家族別の生存率を追加
name_df_train = df.iloc[:,2]
survive_array = df.iloc[:,0]

name_df = name_df_train

name_array = []
name_count = []
name_survive_count = []
name_survive_ratio = []

name_i = 0

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    if name_split_first in name_array:
        name_count[name_array.index(name_split_first)] = name_count[name_array.index(name_split_first)] + 1
        name_survive_count[name_array.index(name_split_first)] = name_survive_count[name_array.index(name_split_first)] + survive_array[name_i]                            
    else:
        name_array.append(name_split_first)
        name_count.append(1)
        name_survive_count.append(survive_array[name_i])
    name_i = name_i + 1
        
count = 0
for i in name_count:
    if name_count[count] == 1:
        name_survive_ratio.append(0)
    else:
        name_survive_ratio.append(name_survive_count[count]/(name_count[count]))
    count = count + 1
    
survive_ratio_array = []    

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    survive_ratio_array.append(name_survive_ratio[name_array.index(name_split_first)])

df['familysurviveratio'] = np.array(survive_ratio_array)

In [5]:
df = df.drop(["Survived","SibSp","Parch"],axis=1)

In [6]:
df.head()

,Pclass,Name,Sex,Ticket,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
0,3,"Braund, Mr. Owen Harris",0,A/5 21171,7.2500,0.0,2,22,1,0.0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,PC 17599,71.2833,1.0,2,38,1,0.0
2,3,"Heikkinen, Miss. Laina",1,STON/O2. 3101282,7.9250,0.0,1,26,2,0.0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,113803,53.1000,0.0,2,35,1,0.5
4,3,"Allen, Mr. William Henry",0,373450,8.0500,0.0,1,35,1,0.5


In [7]:
df = df.drop(["Embarked","Name","Ticket"],axis=1)

In [8]:
df = df.drop(["honor","Fare"],axis=1)

In [9]:
df.head()

,Pclass,Sex,FamilySize,Age,familysurviveratio
0,3,0,2,22,0.0
1,1,1,2,38,0.0
2,3,1,1,26,0.0
3,1,1,2,35,0.5
4,3,0,1,35,0.5


In [10]:
X= df.as_matrix()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [18]:
mod = xgb.XGBClassifier()
mod.fit(X_train, y_train)
print('train accuracy: %.3f' % mod.score(X_train, y_train))
print('test  accuracy: %.3f' % mod.score(X_test, y_test))

train accuracy: 0.888
test  accuracy: 0.862


In [19]:
#テストデータにも同様の処理をする
df2 = pd.read_csv("test.csv")
df2 = df2.drop(["PassengerId","Cabin"], axis = 1)
df2 = df2.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
df2 = df2.fillna(value = {'Embarked': 1})

#敬称ごとに変数を追加
honor1 = []
for i in df2["Name"]:
    if "Master" in i :
        honor1.append(0)
    elif "Mr" in i :
        honor1.append(1)
    elif "Miss" in i :
        honor1.append(2)
    elif "Mrs" in i :
        honor1.append(3)
    else:
        honor1.append("4")
df2['honor'] = np.array(honor1)

#家族の人数を追加
df2["FamilySize"] = df2["SibSp"] + df2["Parch"] + 1

#年齢の欠損を敬称別の中央値で埋める。
df2 = df2.fillna(value = {"Age":-1})
a = df2.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df2[df2["honor"]==i[1]]["Age"].median()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df2 = df2.drop(["Age","honor"], axis = 1)
df2 = pd.concat([df2, frame], axis=1)

In [20]:
name_df_test = df2.iloc[:,1]
survive_ratio_test =[]
for name in name_df_test:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')                                         
    if name_split_first in name_array:
        survive_ratio_test.append(name_survive_ratio[name_array.index(name_split_first)])
    else:
        survive_ratio_test.append(0)
df2= df2.drop(["Name","Ticket"], axis = 1)        
df2['familysurviveratio'] = np.array(survive_ratio_test)


In [21]:
df2.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
0,3,0,0,0,7.8292,2,1,34.5,1,0.75
1,3,1,1,0,7.0000,0,2,47,1,0.00
2,2,0,0,0,9.6875,2,1,62,1,0.00
3,3,0,0,0,8.6625,0,1,27,1,0.00
4,3,1,1,1,12.2875,0,3,22,1,0.00


In [22]:
df2 = df2.drop(["SibSp","Parch","Fare","Embarked","honor"],axis =1)

In [23]:
df2[:10]

,Pclass,Sex,FamilySize,Age,familysurviveratio
0,3,0,1,34.5,0.750000
1,3,1,2,47,0.000000
2,2,0,1,62,0.000000
3,3,0,1,27,0.000000
4,3,1,3,22,0.000000
5,3,0,1,14,0.000000
6,3,1,1,30,0.000000
7,2,0,3,26,1.000000
8,3,1,1,18,0.000000
9,3,0,3,21,0.333333


In [26]:
x = df2.as_matrix()

In [27]:
mod2 = xgb.XGBClassifier()
mod2.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [28]:
y1 = mod2.predict(x)

In [30]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y1})
dfnew2.to_csv("pred30.csv") 

In [57]:
mod2 = xgb.XGBClassifier()
mod2.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [59]:
y2 = mod2.predict(x)
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y2})
dfnew2.to_csv("pred29.csv") 

In [60]:
mod3 = xgb.XGBClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [61]:
mod3.fit(X_train,y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [ ]:
y2 = mod3.predict(x)
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y2})
dfnew2.to_csv("pred29.csv") 

In [31]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest.fit(X_train, y_train)
print('Train score: {}'.format(forest.score(X_train, y_train)))
print('Test score: {}'.format(forest.score(X_test, y_test)))

Train score: 0.8780096308186196
Test score: 0.8656716417910447


In [32]:
y3 = forest.predict(x)
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y3})
dfnew2.to_csv("pred31.csv") 

In [33]:
forest2 = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest2.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [34]:
y4 = forest.predict(x)
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y4})
dfnew2.to_csv("pred32.csv") 